## PGA Draftkings Notebook
Use [PGA Website](https://www.pgatour.com/tournaments/schedule.html) to look up tournament info and fill out first USER INPUT block below. (Fetch/XHR from Network tab, Paylod sub-tab)

Looks like the tournament ID is also in the address bar.

### User Input

In [ ]:
# === USER INPUTS ===
# Old Tournament
old_tournament_name = "John Deere Classic"
tournament_date = "7/6/2025"  # Ending date of tournament
old_course = "TPC Deere Run"
tournament_id = "R2025030"  # Tournament ID from the PGA Tour API - Also visible in the URL of the tournament page

# New Tournament
new_tournament_name = "Genesis Scottish Open"
new_ending_date = "7/13/2025"
new_course = "The Renaissance Club"
new_season = 2025


# === LIBRARIES AND VARIABLES ===
# Import necessary libraries
import requests
import pandas as pd
from datetime import datetime
import sqlite3 as sql
import numpy as np
from numpy import nan
import os
import importlib
import utils.db_utils
from utils.db_utils import TOURNAMENT_NAME_MAP, PLAYER_NAME_MAP

# === TOURNAMENT CONFIG ===
tournament_config = {
    "old": {
        "name": old_tournament_name,
        "date": tournament_date,
        "course": old_course,
        "id": tournament_id
    },
    "new": {
        "name": new_tournament_name,
        "course": new_course,
        "season": new_season,
        "ending_date": pd.to_datetime(new_ending_date, format="%m/%d/%Y"),
        "quoted_course": f'"{new_course}"',
        "quoted_name": f"'{new_tournament_name}'"
    }
}

this_week_key = str(tournament_config["new"]["ending_date"].date())

# Create a minimal one-row history DataFrame
this_week_history = pd.DataFrame([{
    "SEASON": tournament_config["new"]["season"],
    "TOURNAMENT": tournament_config["new"]["name"],
    "ENDING_DATE": tournament_config["new"]["ending_date"],
    "COURSE": tournament_config["new"]["course"],
    "TOURN_ID": None
}])

### Update Database

#### Old Tournament

In [13]:
importlib.reload(utils.db_utils)  # Only needed if you're actively editing db_utils.py
from utils.db_utils import update_tournament_results

# Change these each year!!
season = 2025
year = 20250  # Unique GraphQL year distinguishing number in case of multiple per year

# Run the update
db_path = "data/golf.db"  # Or use os.path.join("data", "golf.db")
tournDf = update_tournament_results(tournament_config, db_path, season, year)

# Show just the most recent tournament added for confirmation
from sqlalchemy import create_engine

engine = create_engine(f"sqlite:///{db_path}")

query = f"""
SELECT *
FROM tournaments
WHERE TOURN_ID = '{tournament_config['old']['id']}'
  AND ENDING_DATE = '{datetime.strptime(tournament_config['old']['date'], '%m/%d/%Y').date()}'
"""

recent = pd.read_sql(query, engine)
engine.dispose()
recent.sort_values(by='FINAL_POS').head()

📦 Fetching results for tournament ID R2025030 (John Deere Classic), year: 20250
ℹ️ Tournament 'John Deere Classic' already exists — no new data inserted.


,SEASON,ENDING_DATE,TOURN_ID,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,ROUNDS:1,ROUNDS:2,ROUNDS:3,ROUNDS:4,OFFICIAL_MONEY,FEDEX_CUP_POINTS
0,2025,2025-07-06,R2025030,John Deere Classic,TPC Deere Run,Brian Campbell,1,1,-6,-5,-3,-4,"$1,512,000.00",500.000
1,2025,2025-07-06,R2025030,John Deere Classic,TPC Deere Run,Emiliano Grillo,2,2,-6,-5,-3,-4,"$915,600.00",300.000
2,2025,2025-07-06,R2025030,John Deere Classic,TPC Deere Run,Kevin Roy,T3,3,-4,-6,-1,-6,"$495,600.00",162.500
3,2025,2025-07-06,R2025030,John Deere Classic,TPC Deere Run,David Lipsky,T3,3,-7,-4,-3,-3,"$495,600.00",162.500
4,2025,2025-07-06,R2025030,John Deere Classic,TPC Deere Run,Carson Young,T5,5,-4,-3,-2,-7,"$278,250.00",90.000


#### Stats

In [14]:
importlib.reload(utils.db_utils)
from utils.db_utils import update_season_stats  # <- This line is essential

# Change these each year!!
statsYear = 2025

stats_df = update_season_stats(statsYear, db_path)
stats_df.head()

✅ Overwrote stats for season 2025 with 998 rows.


,PLAYER,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,...,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,SEASON
0,A.J. Ewart,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,624,.1888,2025
1,Aaron Baddeley,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,488,.2726,2025
2,Aaron Cockerill,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,327,.4381,2025
3,Aaron Rai,27.0,0.769,22.0,0.407,46.0,0.280,64.0,0.082,117.0,...,287.3,1.0,74.04%,21.0,69.44%,111.0,58.77%,34,2.6185,2025
4,Aaron Wilkin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,852,.1117,2025


#### Odds
Not usually needed for weekly routine.

**Manual Fix! Odds name cleanup (only needed when joins fail)**

Make sure to update the dictionaries in db_utils.py if new names need to be added.

In [30]:
importlib.reload(utils.db_utils)
from utils.db_utils import clean_odds_names, PLAYER_NAME_MAP, TOURNAMENT_NAME_MAP

db_path = "data/golf.db" 
updated_odds = clean_odds_names(db_path, TOURNAMENT_NAME_MAP, PLAYER_NAME_MAP)
updated_odds.head()

ℹ️ No odds rows required name cleanup.


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS,TOURNAMENT_ORIG,PLAYER_ORIG


**Historical Odds Updates**

Only run this when loading in entire year odds at the start of each year or if corrections need to be made.  This will load in the entire year into the database and update it using the dictionary in db_utils.py.

In [31]:
importlib.reload(utils.db_utils)
from utils.db_utils import import_historical_odds

oddsYear = "2022-2023"    # URL segment
season = 2023             # PGA Tour season
db_path = "data/golf.db"

odds_df = import_historical_odds(oddsYear, season, db_path)
odds_df.head()

ℹ️ Historical odds for season 2023 already exist — no new rows added.


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS
4,2023,Fortinet Championship,2022-09-18,Hideki Matsuyama,18/1,18.0
5,2023,Fortinet Championship,2022-09-18,Max Homa,18/1,18.0
6,2023,Fortinet Championship,2022-09-18,Corey Conners,20/1,20.0
7,2023,Fortinet Championship,2022-09-18,Maverick McNealy,25/1,25.0
8,2023,Fortinet Championship,2022-09-18,Taylor Pendrith,25/1,25.0


**Not normally needed**

The code below is a way to troubleshoot the odds function that is in db_utils.py.  If it does not pull the stats correctly, we can run it outside of the db_utils.py file and explore what might be wrong with it, iterating until it works, and then using that fix to update db_utils.py.  This is not normally needed, but wanted to leave this in place as a reference because this odds website can be tricky.

In [32]:
import pandas as pd
import numpy as np
import requests
import re
from datetime import datetime
from io import StringIO

# === USER INPUT ===
oddsYear = "2020-2021"    # URL segment
season = 2021        # PGA Tour season

url = f"http://golfodds.com/archives-{oddsYear}.html"
response = requests.get(url)
tables = pd.read_html(StringIO(response.text))
# raw_df = tables[5]  # the actual table of interest
# Find the largest 2-column table that contains at least some odds-like strings
raw_df = None
for tbl in tables:
    if tbl.shape[1] == 2 and tbl.shape[0] > 50:  # Rough filter
        sample = tbl.iloc[:, 1].astype(str).str.contains(r"\d+/\d+").sum()
        if sample > 5:
            raw_df = tbl
            break

if raw_df is None:
    raise ValueError("❌ Could not find valid odds table on the page.")

# === STEP 1: Initial clean-up ===
df = raw_df.dropna(how="all").reset_index(drop=True)
df.columns = ["PLAYER", "ODDS"]

# 🔧 Clean up non-breaking spaces and extra whitespace
df["PLAYER"] = (
    df["PLAYER"]
    .astype(str)
    .str.replace("\xa0", " ", regex=False)
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
)

df.insert(loc=0, column="SEASON", value=season)
df.insert(loc=1, column="TOURNAMENT", value=np.nan)
df.insert(loc=2, column="ENDING_DATE", value=np.nan)

# === STEP 2: Helper function for parsing date strings ===
def parse_ending_date(text):
    import re
    from datetime import datetime

    # Normalize whitespace and symbols
    text = (
        text.replace("\u2013", "-")
            .replace("–", "-")
            .replace("\xa0", " ")
    )
    text = re.sub(r"\bSept(?!ember)\b", "Sep", text)

    # ✅ Fix typo: "Match" → "March" only when used in a date context
    text = re.sub(r"\bMatch(?=\s+\d{1,2}\s*[-–]\s*\d{1,2},\s*\d{4})", "March", text)

    # Pattern 1: "July 30 - August 2, 2015" or "Oct 29 - Nov 1, 2015"
    match = re.search(r"(\w+)\s\d+\s*-\s*(\w+)\s(\d+),\s(\d{4})", text)
    if match:
        month2, day2, year = match.group(2), match.group(3), match.group(4)
        for fmt in ["%B %d, %Y", "%b %d, %Y"]:
            try:
                return datetime.strptime(f"{month2} {day2}, {year}", fmt).date()
            except ValueError:
                continue

    # Pattern 2: "November 21-24, 2024"
    match = re.search(r"(\w+)\s\d+-\d+,\s(\d{4})", text)
    if match:
        month, year = match.group(1), match.group(2)
        day = re.search(r"(\d+)-(\d+)", text).group(2)
        for fmt in ["%B %d, %Y", "%b %d, %Y"]:
            try:
                return datetime.strptime(f"{month} {day}, {year}", fmt).date()
            except ValueError:
                continue

    # Pattern 3: "Sunday, October 20, 2019"
    try:
        return datetime.strptime(text.strip(), "%A, %B %d, %Y").date()
    except ValueError:
        pass

    # Pattern 4: "October 20, 2019"
    try:
        return datetime.strptime(text.strip(), "%B %d, %Y").date()
    except ValueError:
        pass

    return None

last_tourn_name = None
last_date = None

# === STEP 3: Iterate block by block ===
final_rows = []
i = 0
last_tourn_name = None
last_end_date = None

while i < len(df) - 4:
    player_i = str(df.loc[i, "PLAYER"])
    player_i2 = str(df.loc[i + 2, "PLAYER"])
    player_i3 = str(df.loc[i + 3, "PLAYER"]).lower()

    # Detect start of a new tournament block
    is_header = (
        pd.isna(df.loc[i, "ODDS"]) and
        pd.isna(df.loc[i + 1, "ODDS"]) and (
            re.search(r"\w+\s\d+\s*[-–]\s*(\w+\s)?\d+,\s\d{4}", player_i2) or
            re.search(r"(Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday),?\s+\w+\s\d{1,2},\s\d{4}", player_i2)
        )
    )

    if is_header:
        tourn_name = player_i.strip()
        end_date = parse_ending_date(player_i2)

        # Skip cancelled or empty blocks
        if "cancelled" in player_i3:
            print(f"⚠️ Skipping cancelled tournament: {tourn_name} — {end_date}")
            i += 4
            continue

        # Avoid duplicate block processing
        if tourn_name == last_tourn_name and end_date == last_end_date:
            i += 1
            continue

        print(f"📍 Detected: {tourn_name} — Ending: {end_date}")
        last_tourn_name = tourn_name
        last_end_date = end_date
        i += 4  # Skip header lines

        # Collect all player rows until next header block
        while i < len(df) - 2:
            next_i2 = str(df.loc[i + 2, "PLAYER"])
            is_next_header = (
                pd.isna(df.loc[i, "ODDS"]) and
                pd.isna(df.loc[i + 1, "ODDS"]) and (
                    re.search(r"\w+\s\d+\s*[-–]\s*(\w+\s)?\d+,\s\d{4}", next_i2) or
                    re.search(r"(Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday),?\s+\w+\s\d{1,2},\s\d{4}", next_i2)
                )
            )
            if is_next_header:
                break

            if pd.notna(df.loc[i, "ODDS"]):
                row = df.loc[i].copy()
                row["TOURNAMENT"] = tourn_name
                row["ENDING_DATE"] = end_date
                final_rows.append(row)
            i += 1
    else:
        i += 1

# === STEP 4: Create cleaned DataFrame ===
clean_df = pd.DataFrame(final_rows)

# ✅ Prevent crash if nothing was parsed
if clean_df.empty or "PLAYER" not in clean_df.columns:
    print(f"⚠️ No valid tournament blocks detected for season {season} ({oddsYear})")
    final_df = pd.DataFrame()  # Safe fallback
else:
    # Remove winner tag
    clean_df["PLAYER"] = clean_df["PLAYER"].str.replace(r"\s\*Winner\*", "", regex=True)

    # Clean odds to numeric
    clean_df["VEGAS_ODDS"] = (
        clean_df["ODDS"]
        .str.replace(",", "")
        .str.extract(r"(\d+)/(\d+)")
        .astype(float)
        .apply(lambda x: x[0] / x[1], axis=1)
    )

    # Final output with source index for debugging
    final_df = clean_df[
        ["SEASON", "TOURNAMENT", "ENDING_DATE", "PLAYER", "ODDS", "VEGAS_ODDS"]
    ].reset_index(drop=True)

    # Drop non-standard team events (e.g., Presidents Cup, Ryder Cup)
    drop_terms = ["Presidents Cup", "Ryder Cup"]
    final_df = final_df[~final_df["TOURNAMENT"].str.contains("|".join(drop_terms), case=False, na=False)]

    display(final_df.head())



📍 Detected: Safeway Open — Ending: 2020-09-13
📍 Detected: US Open — Ending: 2020-09-20
📍 Detected: R & C Championship — Ending: 2020-09-27
📍 Detected: at Big Cedar Lodge - — Ending: 2020-09-22
📍 Detected: Sanderson Farms Champ — Ending: 2020-10-04
📍 Detected: Shriners H for C Open — Ending: 2020-10-11
📍 Detected: The CJ Cup — Ending: 2020-10-18
📍 Detected: ZOZO CHAMPIONSHIP — Ending: 2020-10-25
📍 Detected: Bermuda Championship — Ending: 2020-11-01
📍 Detected: Vivint Houston Open — Ending: 2020-11-08
📍 Detected: The Masters — Ending: 2020-11-15
📍 Detected: The RSM Classic — Ending: 2020-11-22
📍 Detected: Champions for Change — Ending: 2020-11-27
📍 Detected: Mayakoba Golf Classic — Ending: 2020-12-06
📍 Detected: QBE Shootout — Ending: 2020-12-13
📍 Detected: Sentry Tourn of Champions — Ending: 2021-01-10
📍 Detected: Sony Open in Hawaii — Ending: 2021-01-17
📍 Detected: The American Express — Ending: 2021-01-24
📍 Detected: Abu Dhabi HSBC Champ — Ending: 2021-01-24
📍 Detected: Farmers Insura

,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS
0,2021,Safeway Open,2020-09-13,Phil Mickelson,20/1,20.0
1,2021,Safeway Open,2020-09-13,Si Woo Kim,20/1,20.0
2,2021,Safeway Open,2020-09-13,Brendan Steele,20/1,20.0
3,2021,Safeway Open,2020-09-13,Shane Lowry,25/1,25.0
4,2021,Safeway Open,2020-09-13,Sergio Garcia,30/1,30.0


In [33]:
from datetime import datetime, date
# ✅ Check for non-date types in ENDING_DATE
non_dates = final_df[~final_df["ENDING_DATE"].apply(lambda x: isinstance(x, date))]

print(f"🧪 Rows with invalid ENDING_DATE values: {len(non_dates)}")
display(non_dates.head(10))



🧪 Rows with invalid ENDING_DATE values: 0


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS


In [34]:
dupes = final_df.duplicated(subset=["SEASON", "TOURNAMENT", "ENDING_DATE", "PLAYER"], keep=False)

print(f"🚨 Duplicate primary keys in final_df: {dupes.sum()}")
display(final_df[dupes].sort_values(by=["SEASON", "TOURNAMENT", "PLAYER"]))

🚨 Duplicate primary keys in final_df: 0


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS


#### Misc. Cleaning
Not normally needed

In [26]:
importlib.reload(utils.db_utils)
from utils.db_utils import clean_player_names_in_table, PLAYER_NAME_MAP

db_path = "data/golf.db"

# Run for all relevant tables
clean_player_names_in_table(db_path, "tournaments", PLAYER_NAME_MAP)
clean_player_names_in_table(db_path, "stats", PLAYER_NAME_MAP)
clean_player_names_in_table(db_path, "odds", PLAYER_NAME_MAP)

ℹ️ No player names needed updates in 'tournaments'.
ℹ️ No player names needed updates in 'stats'.
ℹ️ No player names needed updates in 'odds'.


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS,PLAYER_ORIG


## Historical Data

### Pull Relevant Seasons
Do a check to see when this course or tournament have been historically played.

In [15]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_combined_history_seasons

# === USER INPUT ===
seasons = list(range(2016, 2025))  # Adjust as needed
db_path = "data/golf.db"

# Pull course and tournament from config
n_course = tournament_config["new"]["course"]
n_tourn = tournament_config["new"]["name"]

# Fetch relevant history
history_df = get_combined_history_seasons(db_path, course=n_course, tournament=n_tourn, allowed_seasons=seasons)
history_df.head(20)


ℹ️ Found 3 relevant tournaments from course or tournament name.


,SEASON,COURSE,TOURN_ID,TOURNAMENT,ENDING_DATE
0,2022,The Renaissance Club,521,Genesis Scottish Open,2022-07-10
78,2023,The Renaissance Club,R2023541,Genesis Scottish Open,2023-07-16
234,2024,The Renaissance Club,R2024541,Genesis Scottish Open,2024-07-14


### Cut Percentage and FedEx Points
Use a rolling-window approach to look at the most recent cut percentage and how many FedEx cup points have been accumulated recently. This will intentionally not match the PGA Tour stats that start over every year, but will have the same amount of data all the time.  We also add a new feature called Form Density which divides the FedEx Cup Points by the Total Events.

In [16]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_cut_and_fedex_history

cuts = get_cut_and_fedex_history("data/golf.db", history_df, window_months=9)
# cuts["2024-05-12"].head(20)

for end_date, df in cuts.items():
    print(f"\n📆 {end_date} — {df['TOURNAMENT'].iloc[0]} ({len(df)} players)")
    display(df.head(3))


📆 2022-07-10 — Genesis Scottish Open (525 players)


,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,A.J. Ewart,1,0,0.00,0.0,0.00,0,2022-07-10,Genesis Scottish Open
1,Aaron Baddeley,5,2,45.94,40.0,9.19,1,2022-07-10,Genesis Scottish Open
2,Aaron Beverly,1,0,0.00,0.0,0.00,0,2022-07-10,Genesis Scottish Open



📆 2023-07-16 — Genesis Scottish Open (539 players)


,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,Aaron Baddeley,16,11,349.536,68.8,21.85,1,2023-07-16,Genesis Scottish Open
1,Aaron Beverly,1,1,0.000,100.0,0.00,1,2023-07-16,Genesis Scottish Open
2,Aaron Cockerill,1,0,0.000,0.0,0.00,0,2023-07-16,Genesis Scottish Open



📆 2024-07-14 — Genesis Scottish Open (507 players)


,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,Aaron Baddeley,14,10,184.725,71.4,13.19,1,2024-07-14,Genesis Scottish Open
1,Aaron Cockerill,1,0,0.000,0.0,0.00,0,2024-07-14,Genesis Scottish Open
2,Aaron Rai,18,15,808.257,83.3,44.90,9,2024-07-14,Genesis Scottish Open


### Recent Form

In [17]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_recent_avg_finish

recent_form = get_recent_avg_finish("data/golf.db", history_df, window_months=9)

# Example preview
for date, df in recent_form.items(): 
    print(f"\n📆 {date} — {df['TOURNAMENT'].iloc[0]} ({len(df)} players)")
    display(df.head(3))


📆 2022-07-10 — Genesis Scottish Open (525 players)


,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Chris Gotterup,1,4.0,5.77,2022-07-10,Genesis Scottish Open
1,Haotong Li,1,12.0,17.31,2022-07-10,Genesis Scottish Open
2,Martin Contini,1,16.0,23.08,2022-07-10,Genesis Scottish Open



📆 2023-07-16 — Genesis Scottish Open (539 players)


,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Brooks Koepka,3,6.7,4.83,2023-07-16,Genesis Scottish Open
1,Scottie Scheffler,18,7.6,2.58,2023-07-16,Genesis Scottish Open
2,Ryo Hisatsune,1,12.0,17.31,2023-07-16,Genesis Scottish Open



📆 2024-07-14 — Genesis Scottish Open (507 players)


,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Bryson DeChambeau,3,3.0,2.16,2024-07-14,Genesis Scottish Open
1,Scottie Scheffler,16,6.3,2.22,2024-07-14,Genesis Scottish Open
2,Xander Schauffele,16,12.9,4.55,2024-07-14,Genesis Scottish Open


### Course History

In [18]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_course_history

# Filter history_df for only the course we're targeting
target_course = tournament_config["new"]["course"]
course_df = history_df[history_df["COURSE"] == target_course]
course_hist = get_course_history("data/golf.db", course_df)

# View example
for date, df in course_hist.items():
    if not df.empty:
        print(f"\n🏌️‍♂️ Course history for {df['TOURNAMENT'].iloc[0]} on {date}")
        display(df.head(3))


🏌️‍♂️ Course history for Genesis Scottish Open on 2023-07-16


,PLAYER,TOTAL_EVENTS_PLAYED,COURSE_HISTORY,adj_ch,ENDING_DATE,COURSE,TOURNAMENT
0,Aaron Wise,1,5.0,7.21,2023-07-16,The Renaissance Club,Genesis Scottish Open
1,Abraham Ancer,1,14.0,20.20,2023-07-16,The Renaissance Club,Genesis Scottish Open
2,Adam Scott,1,5.0,7.21,2023-07-16,The Renaissance Club,Genesis Scottish Open



🏌️‍♂️ Course history for Genesis Scottish Open on 2024-07-14


,PLAYER,TOTAL_EVENTS_PLAYED,COURSE_HISTORY,adj_ch,ENDING_DATE,COURSE,TOURNAMENT
0,Aaron Baddeley,1,90.0,129.84,2024-07-14,The Renaissance Club,Genesis Scottish Open
1,Aaron Rai,1,90.0,129.84,2024-07-14,The Renaissance Club,Genesis Scottish Open
2,Aaron Wise,1,5.0,7.21,2024-07-14,The Renaissance Club,Genesis Scottish Open


In [19]:
#  DEBUGGING: Check for empty DataFrames or missing columns
for date_key, df in course_hist.items():
    if df.empty:
        print(f"ℹ️ Empty course_hist for date {date_key}")
    elif "PLAYER" not in df.columns:
        print(f"❌ Missing 'PLAYER' column in course_hist[{date_key}]. Columns present: {df.columns.tolist()}")
        display(df.head())

ℹ️ Empty course_hist for date 2022-07-10


## Training Dataset

In [20]:
pd.set_option("display.max_columns", None)   # Show all columns
importlib.reload(utils.db_utils)
from utils.db_utils import build_training_rows
training_df = build_training_rows(
    db_path,
    history_df,
    cuts,
    recent_form,
    course_hist,
)
training_df.head(10)
# training_df.info()
# training_df.isna().sum().sort_values(ascending=False)

,SEASON,ENDING_DATE,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,TOP_20
0,2022,2022-07-10,Genesis Scottish Open,The Renaissance Club,Rory McIlroy,1,1,2.0,1.619,3.0,0.771,14.0,0.626,43.0,0.222,16.0,0.495,4.0,26.11,51.0,3.03,3.0,3.95,3.0,4.44,23.0,129.0,2.0,321.3,127.0,58.06,12.0,69.91,30.0,62.77,2.0,9.07,NaN,91.7,1893.88,157.82,7.0,17.1,6.67,NaN,NaN,1
1,2022,2022-07-10,Genesis Scottish Open,The Renaissance Club,Collin Morikawa,2,2,17.0,1.066,36.0,0.361,3.0,0.854,152.0,-0.149,131.0,-0.088,31.0,23.06,31.0,3.02,17.0,3.98,53.0,4.59,9.0,112.0,83.0,301.8,29.0,65.27,30.0,68.61,94.0,59.59,9.0,6.33,25.0,83.3,1015.59,84.63,1.0,32.9,12.83,NaN,NaN,1
2,2022,2022-07-10,Genesis Scottish Open,The Renaissance Club,Rickie Fowler,T3,3,101.0,0.092,114.0,-0.022,150.0,-0.161,31.0,0.274,161.0,-0.253,165.0,19.64,83.0,3.05,183.0,4.09,106.0,4.64,48.0,158.0,27.0,309.4,131.0,57.91,186.0,62.21,152.0,56.94,185.0,0.96,125.0,58.8,300.54,17.68,0.0,60.6,20.97,NaN,NaN,1
3,2022,2022-07-10,Genesis Scottish Open,The Renaissance Club,Keith Mitchell,T3,3,29.0,0.865,4.0,0.761,111.0,0.024,83.0,0.079,55.0,0.209,31.0,23.06,18.0,3.01,38.0,4.00,12.0,4.52,2.0,91.0,18.0,311.8,73.0,61.76,57.0,67.64,20.0,63.67,59.0,1.97,80.0,83.3,833.39,46.30,5.0,35.3,11.99,NaN,NaN,1
4,2022,2022-07-10,Genesis Scottish Open,The Renaissance Club,Sam Burns,T5,5,32.0,0.838,86.0,0.067,18.0,0.554,45.0,0.217,10.0,0.553,10.0,24.44,31.0,3.02,17.0,3.98,7.0,4.50,83.0,186.0,24.0,309.6,162.0,55.38,26.0,68.68,92.0,59.65,12.0,5.67,25.0,64.7,1595.19,93.83,0.0,39.6,13.70,NaN,NaN,1
5,2022,2022-07-10,Genesis Scottish Open,The Renaissance Club,Talor Gooch,T5,5,21.0,1.017,140.0,-0.138,13.0,0.653,3.0,0.502,100.0,0.005,12.0,23.94,1.0,2.96,38.0,4.00,5.0,4.48,25.0,133.0,64.0,304.4,69.0,61.88,16.0,69.33,63.0,60.71,35.0,2.70,NaN,72.2,1101.31,61.18,0.0,38.6,13.11,NaN,NaN,1
6,2022,2022-07-10,Genesis Scottish Open,The Renaissance Club,Adam Scott,T5,5,66.0,0.340,68.0,0.158,53.0,0.256,136.0,-0.075,41.0,0.279,30.0,23.07,161.0,3.10,25.0,3.99,29.0,4.56,70.0,178.0,15.0,312.4,163.0,55.34,55.0,67.67,133.0,57.76,30.0,2.86,NaN,84.6,408.00,31.38,2.0,41.5,15.73,NaN,NaN,1
7,2022,2022-07-10,Genesis Scottish Open,The Renaissance Club,Aaron Wise,T5,5,25.0,0.952,31.0,0.387,21.0,0.489,85.0,0.076,81.0,0.100,22.0,23.39,4.0,2.98,25.0,3.99,24.0,4.55,44.0,154.0,39.0,308.3,115.0,59.15,21.0,69.06,31.0,62.63,37.0,2.69,NaN,72.2,846.11,47.01,0.0,42.7,14.50,NaN,NaN,1
8,2022,2022-07-10,Genesis Scottish Open,The Renaissance Club,Harry Higgs,T9,9,188.0,-1.039,176.0,-0.350,187.0,-0.600,139.0,-0.089,53.0,0.215,190.0,18.13,105.0,3.06,183.0,4.09,170.0,4.70,156.0,240.0,83.0,301.8,157.0,55.52,189.0,61.39,67.0,60.61,230.0,0.81,NaN,47.8,221.15,9.62,0.0,67.0,21.08,NaN,NaN,1
9,2022,2022-07-10,Genesis Scottish Open,The Renaissance Club,Sungjae Im,T9,9,9.0,1.342,8.0,0.651,45.0,0.308,12.0,0.383,37.0,0.326,24.0,23.3,18.0,3.01,3.0,3.95,10.0,4.51,4.0,98.0,71.0,303.7,27.0,65.51,9.0,70.83,5.0,66.02,18.0,4.21,40.0,81.2,1175.61,73.48,0.0,30.4,10.73,NaN,NaN,1


In [21]:
# Play with different years to see if I notice anything wrong (check Odds)
training_df[training_df["SEASON"] == 2024].head()

,SEASON,ENDING_DATE,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,TOP_20
235,2024,2024-07-14,Genesis Scottish Open,The Renaissance Club,Robert MacIntyre,1,1,55.0,0.404,38.0,0.289,112.0,-0.051,48.0,0.167,36.0,0.321,90.0,22.30%,74.0,3.04,26.0,3.98,44.0,4.54,134.0,202.0,63.0,304.9,139.0,56.64%,115.0,65.97%,16.0,64.85%,16.0,3.6699,40.0,43.8,927.752,57.98,0.0,61.6,21.74,2.0,2.89,1
236,2024,2024-07-14,Genesis Scottish Open,The Renaissance Club,Adam Scott,2,2,38.0,0.540,33.0,0.322,87.0,0.114,67.0,0.104,29.0,0.398,79.0,22.54%,38.0,3.01,57.0,4.00,44.0,4.54,75.0,170.0,49.0,307.1,121.0,58.28%,102.0,66.35%,45.0,62.09%,20.0,3.2584,80.0,87.5,693.009,43.31,4.0,33.6,11.86,47.5,43.24,1
237,2024,2024-07-14,Genesis Scottish Open,The Renaissance Club,Romain Langasque,3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,85.0,1.4115,150.0,NaN,NaN,NaN,NaN,NaN,NaN,25.0,36.07,1
238,2024,2024-07-14,Genesis Scottish Open,The Renaissance Club,Aaron Rai,T4,4,9.0,1.131,36.0,0.303,8.0,0.676,56.0,0.152,83.0,0.054,99.0,22.10%,38.0,3.01,10.0,3.97,44.0,4.54,53.0,153.0,152.0,293.8,1.0,72.02%,7.0,71.58%,33.0,63.18%,21.0,3.2294,40.0,83.3,808.257,44.90,9.0,35.8,12.16,90.0,129.84,1
239,2024,2024-07-14,Genesis Scottish Open,The Renaissance Club,Collin Morikawa,T4,4,8.0,1.150,16.0,0.435,46.0,0.283,4.0,0.432,61.0,0.165,17.0,24.30%,21.0,3.00,5.0,3.96,53.0,4.55,36.0,140.0,136.0,296.0,4.0,70.57%,135.0,65.04%,9.0,65.68%,4.0,6.1022,16.0,88.2,2232.296,131.31,10.0,24.2,8.37,2.0,2.89,1


### Normalization
***Fix Historical Odds***

Many names do not have historical odds but rather are part of the "field." So giving these "NaN" values the average odds (as I would for missing stats) isn't a good approximation of reality – they are typically the "field" because they are not notable and have poor odds individually.

There are also some crazy odds numbers occasionally (1000/1 or 3000/1) that don't happen all that much and are making that tail too long.

This cell assigns odds of 1000/1 for anyone missing and clips anything larger to 1000/1 to normalize the data better and assign the missing odds more appropriately where they belong.

***OWGR Adjustment***

The NaN's for OWGR are similarly bad players.  So we should assign these the worst ranking instead of the mean and clip it at 1000 just in case there are ever outliers.

For the OWGR score, it should match the lowest score in the dataset to assign that person the same as the worst.

***Recent Form Adjustment***

The NaN's for Recent Form mean that this player has not played any tournaments in the lookback period (9 months).  Similar to Odds and OWGR, I want to punish those that don't play often in my model.  Set these to 90 (i.e. like they miss a lot of cuts - because if they aren't playing they aren't good enough to make it to these tournaments very frequently).  The adj_form feature will also need to be updated here with the new data.

***FedEx Cup Point Adjustment***

The NaN's for FedEx Cup Points means there is no data.  Therefore they should be 0.

In [11]:
# === VEGAS ODDS ===
training_df["VEGAS_ODDS"] = training_df["VEGAS_ODDS"].fillna(1000).clip(upper=1000)

# === OWGR and OWGR_RANK ===
owgr_min = training_df["OWGR"].min(skipna=True)
training_df["OWGR"] = training_df["OWGR"].fillna(owgr_min)
training_df["OWGR_RANK"] = training_df["OWGR_RANK"].fillna(1000).astype(float).clip(upper=1000)

# === RECENT FORM and adj_form ===
training_df["RECENT_FORM"] = training_df["RECENT_FORM"].fillna(90)
if "TOTAL_EVENTS_PLAYED" in training_df.columns:
    training_df["adj_form"] = (
        training_df["RECENT_FORM"] / np.log1p(training_df["TOTAL_EVENTS_PLAYED"])
    ).round(2)

# === FEDEX CUP POINTS ===
training_df["FEDEX_CUP_POINTS"] = training_df["FEDEX_CUP_POINTS"].fillna(0)

training_df.head(5)


,SEASON,ENDING_DATE,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,TOP_20
0,2022,2022-07-10,Genesis Scottish Open,The Renaissance Club,Aaron Wise,T5,5,25.0,0.952,31.0,0.387,21.0,0.489,85.0,0.076,81.0,0.100,22.0,23.39,4.0,2.98,25.0,3.99,24.0,4.55,44.0,154.0,39.0,308.3,115.0,59.15,21.0,69.06,31.0,62.63,37.0,2.69,1000.0,72.2,846.11,47.01,0.0,42.7,14.50,NaN,NaN,1
1,2022,2022-07-10,Genesis Scottish Open,The Renaissance Club,Abraham Ancer,T14,14,122.0,-0.026,29.0,0.395,109.0,0.026,189.0,-0.447,62.0,0.165,61.0,21.84,62.0,3.04,25.0,3.99,81.0,4.62,18.0,124.0,120.0,297.1,4.0,70.54,77.0,66.86,64.0,60.69,24.0,3.21,1000.0,68.8,338.30,21.14,1.0,50.0,17.65,NaN,NaN,1
2,2022,2022-07-10,Genesis Scottish Open,The Renaissance Club,Adam Scott,T5,5,66.0,0.340,68.0,0.158,53.0,0.256,136.0,-0.075,41.0,0.279,30.0,23.07,161.0,3.10,25.0,3.99,29.0,4.56,70.0,178.0,15.0,312.4,163.0,55.34,55.0,67.67,133.0,57.76,30.0,2.86,1000.0,84.6,408.00,31.38,2.0,41.5,15.73,NaN,NaN,1
3,2022,2022-07-10,Genesis Scottish Open,The Renaissance Club,Alex Noren,T59,59,113.0,0.026,112.0,-0.019,119.0,0.002,93.0,0.042,18.0,0.457,143.0,20.23,14.0,3.00,76.0,4.02,44.0,4.58,167.0,248.0,100.0,299.8,148.0,56.22,117.0,65.71,12.0,64.56,54.0,2.04,80.0,68.8,444.57,27.79,0.0,47.6,16.80,NaN,NaN,0
4,2022,2022-07-10,Genesis Scottish Open,The Renaissance Club,Branden Grace,T68,68,144.0,-0.290,92.0,0.028,170.0,-0.318,110.0,0.001,188.0,-0.544,177.0,19.23,155.0,3.09,122.0,4.04,157.0,4.69,142.0,225.0,127.0,296.4,98.0,60.14,129.0,65.38,86.0,59.88,155.0,1.08,100.0,41.2,147.03,8.65,0.0,69.6,24.08,NaN,NaN,0


#### Average the NaNs
At this point, the intentional offsetting of NaNs for poor players is over.  I don't want to penalize statistics or course history because you never know where they might land on the spectrum, so for these NaN values, we will take the mean.

In [22]:
# === Clean percentage stats stored as strings like '62.5%' ===
percent_stats = ["SCRAMBLING", "DRIVING_ACCURACY", "BIRDIES", "GIR"]

for col in percent_stats:
    if col in training_df.columns:
        training_df[col] = (
            training_df[col]
            .astype(str)
            .str.replace('%', '', regex=False)
            .replace(['None', 'nan', 'NaN', '--', 'DNP', ''], np.nan)
            .astype(float)
        )

# Select only the numeric columns
numeric_columns = training_df.select_dtypes(include=['float64', 'int64'])

# Fill NaN values with the mean of each column
numeric_columns = numeric_columns.fillna(numeric_columns.mean())

# Update the original dataframe with the filled numeric columns
training_df.update(numeric_columns)

training_df.head()

,SEASON,ENDING_DATE,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,TOP_20
0,2022,2022-07-10,Genesis Scottish Open,The Renaissance Club,Rory McIlroy,1,1,2.0,1.619,3.0,0.771,14.0,0.626,43.0,0.222,16.0,0.495,4.0,26.11,51.0,3.03,3.0,3.95,3.0,4.44,23.0,129.0,2.0,321.3,127.0,58.06,12.0,69.91,30.0,62.77,2.0,9.07,383.960756,91.7,1893.88,157.82,7.0,17.1,6.67,51.160448,70.043209,1
1,2022,2022-07-10,Genesis Scottish Open,The Renaissance Club,Collin Morikawa,2,2,17.0,1.066,36.0,0.361,3.0,0.854,152.0,-0.149,131.0,-0.088,31.0,23.06,31.0,3.02,17.0,3.98,53.0,4.59,9.0,112.0,83.0,301.8,29.0,65.27,30.0,68.61,94.0,59.59,9.0,6.33,25.000000,83.3,1015.59,84.63,1.0,32.9,12.83,51.160448,70.043209,1
2,2022,2022-07-10,Genesis Scottish Open,The Renaissance Club,Rickie Fowler,T3,3,101.0,0.092,114.0,-0.022,150.0,-0.161,31.0,0.274,161.0,-0.253,165.0,19.64,83.0,3.05,183.0,4.09,106.0,4.64,48.0,158.0,27.0,309.4,131.0,57.91,186.0,62.21,152.0,56.94,185.0,0.96,125.000000,58.8,300.54,17.68,0.0,60.6,20.97,51.160448,70.043209,1
3,2022,2022-07-10,Genesis Scottish Open,The Renaissance Club,Keith Mitchell,T3,3,29.0,0.865,4.0,0.761,111.0,0.024,83.0,0.079,55.0,0.209,31.0,23.06,18.0,3.01,38.0,4.00,12.0,4.52,2.0,91.0,18.0,311.8,73.0,61.76,57.0,67.64,20.0,63.67,59.0,1.97,80.000000,83.3,833.39,46.30,5.0,35.3,11.99,51.160448,70.043209,1
4,2022,2022-07-10,Genesis Scottish Open,The Renaissance Club,Sam Burns,T5,5,32.0,0.838,86.0,0.067,18.0,0.554,45.0,0.217,10.0,0.553,10.0,24.44,31.0,3.02,17.0,3.98,7.0,4.50,83.0,186.0,24.0,309.6,162.0,55.38,26.0,68.68,92.0,59.65,12.0,5.67,25.000000,64.7,1595.19,93.83,0.0,39.6,13.70,51.160448,70.043209,1


In [23]:
# Check for NaN values
assert training_df.isna().sum().sum() == 0, "🚨 Still missing values!"

In [24]:
# Inspect which columns have NaN values
training_df.isna().sum().sort_values(ascending=False).head(10)


SEASON         0
ENDING_DATE    0
TOURNAMENT     0
COURSE         0
PLAYER         0
POS            0
FINAL_POS      0
SGTTG_RANK     0
SGTTG          0
SGOTT_RANK     0
dtype: int64

### Check Features for Outliers and General Health (Histograms)
Now that all the data is cleaned, let's look at the distribution graphs.

In [25]:
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# Select numeric columns with >1 unique value, excluding label
df_numeric = training_df.select_dtypes(include=np.number)
df_numeric = df_numeric.loc[:, df_numeric.nunique() > 1]
df_numeric = df_numeric.drop(columns=["TOP_20"], errors="ignore")

# Create subplots
n_cols = 3
n_rows = int(np.ceil(len(df_numeric.columns) / n_cols))
fig = sp.make_subplots(rows=n_rows, cols=n_cols, subplot_titles=df_numeric.columns)

# Add one histogram per feature with smart binning
for i, column in enumerate(df_numeric.columns):
    row = i // n_cols + 1
    col = i % n_cols + 1

    # Compute dynamic bin count
    col_range = df_numeric[column].max() - df_numeric[column].min()
    n_unique = df_numeric[column].nunique()
    nbins = min(50, max(5, int(n_unique / 2))) if col_range < 10 else 50  # tighter for short-range

    fig.add_trace(
        go.Histogram(
            x=df_numeric[column],
            nbinsx=nbins,
            histnorm="density",
            marker=dict(color="#636EFA"),
            opacity=0.8
        ),
        row=row,
        col=col
    )

# Style the figure
fig.update_layout(
    height=300 * n_rows,
    width=1000,
    title_text="Feature Distributions (with Adaptive Binning)",
    template="plotly_dark",
    showlegend=False
)

fig.show()

# Current Week Data

## Import 2025 Field
**Important!**  Save curren't week's `DKSalaries.csv` into Data folder before this step.

In [27]:
importlib.reload(utils.db_utils)
from utils.db_utils import DK_PLAYER_NAME_MAP
from utils.db_utils import standardize_player_names

# Load DraftKings player list
dk = pd.read_csv("data/DKSalaries.csv", usecols=["Name", "Salary"])

# Normalize DK names to PGA naming convention
dk["Name"] = dk["Name"].replace(DK_PLAYER_NAME_MAP)

# Rename for consistency
dk = dk.rename(columns={"Name": "PLAYER", "Salary": "SALARY"})

dk = standardize_player_names(dk)  # ✅ Normalize to match DB

dk.head(10)

,PLAYER,SALARY
0,Scottie Scheffler,13100
1,Rory McIlroy,11400
2,Xander Schauffele,10100
3,Tommy Fleetwood,9900
4,Collin Morikawa,9600
5,Ludvig Aberg,9400
6,Viktor Hovland,9300
7,Justin Thomas,9200
8,Robert Macintyre,9100
9,Sam Burns,9000


In [8]:
print([ord(c) for c in dk.loc[dk["PLAYER"] == "Ludvig Aberg", "PLAYER"].iloc[0]])

[76, 117, 100, 118, 105, 103, 32, 65, 98, 101, 114, 103]


## Vegas Odds
Confirm the [Vegas Odds](http://golfodds.com/weekly-odds.html) page is updated with current tournament.

Note: sometimes this site can give errors and not load correctly.  Just have to do it again later.
Another trick to try is change the url try to run it and then put the url back the way it was.  Worked once, maybe random coincidence.

In [28]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_current_week_odds

odds_current = get_current_week_odds(season=2025, tournament_name=new_tournament_name)
odds_current.head(10)


,SEASON,TOURNAMENT,PLAYER,ODDS,VEGAS_ODDS
1,2025,Genesis Scottish Open,Scottie Scheffler,11/2,5.5
2,2025,Genesis Scottish Open,Rory McIlroy,10/1,10.0
3,2025,Genesis Scottish Open,Tommy Fleetwood,20/1,20.0
4,2025,Genesis Scottish Open,Xander Schauffele,20/1,20.0
5,2025,Genesis Scottish Open,Ludvig Aberg,25/1,25.0
6,2025,Genesis Scottish Open,Collin Morikawa,20/1,20.0
7,2025,Genesis Scottish Open,Robert MacIntyre,25/1,25.0
8,2025,Genesis Scottish Open,Viktor Hovland,30/1,30.0
9,2025,Genesis Scottish Open,Justin Thomas,35/1,35.0
10,2025,Genesis Scottish Open,Sam Burns,35/1,35.0


## Cut Percentage and FedEx Points

In [30]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_cut_and_fedex_history

cuts_rolling = get_cut_and_fedex_history(db_path, this_week_history, window_months=9)
cuts_current = cuts_rolling[this_week_key].copy()
cuts_current.head(10)

,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,A.J. Ewart,1,0,0.000,0.0,0.00,0,2025-07-13,Genesis Scottish Open
1,Aaron Baddeley,15,6,49.088,40.0,3.27,0,2025-07-13,Genesis Scottish Open
2,Aaron Rai,17,13,660.480,76.5,38.85,2,2025-07-13,Genesis Scottish Open
3,Aaron Wise,3,1,6.333,33.3,2.11,0,2025-07-13,Genesis Scottish Open
4,Adam Hadwin,20,13,214.432,65.0,10.72,0,2025-07-13,Genesis Scottish Open
5,Adam Long,1,0,0.000,0.0,0.00,0,2025-07-13,Genesis Scottish Open
6,Adam Schenk,20,9,270.904,45.0,13.55,0,2025-07-13,Genesis Scottish Open
7,Adam Scott,13,11,392.203,84.6,30.17,6,2025-07-13,Genesis Scottish Open
8,Adam Svensson,20,11,135.343,55.0,6.77,1,2025-07-13,Genesis Scottish Open
9,Adrien Dumont de Chassart,6,2,195.000,33.3,32.50,0,2025-07-13,Genesis Scottish Open


## Recent Form

In [31]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_recent_avg_finish

recent_form = get_recent_avg_finish(db_path, this_week_history, window_months=9)
recent_form_current = recent_form[this_week_key].copy()

recent_form_current.head(10)

,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Carlos Ortiz,1,4.0,5.77,2025-07-13,Genesis Scottish Open
1,Scottie Scheffler,15,6.9,2.49,2025-07-13,Genesis Scottish Open
2,Jon Rahm,3,9.7,7.00,2025-07-13,Genesis Scottish Open
3,Bubba Watson,1,14.0,20.20,2025-07-13,Genesis Scottish Open
4,Taisei Shimizu,1,16.0,23.08,2025-07-13,Genesis Scottish Open
5,Ian Gilligan,1,16.0,23.08,2025-07-13,Genesis Scottish Open
6,Stewart Cink,1,17.0,24.53,2025-07-13,Genesis Scottish Open
7,Rory McIlroy,11,19.0,7.65,2025-07-13,Genesis Scottish Open
8,Ren Yonezawa,1,22.0,31.74,2025-07-13,Genesis Scottish Open
9,Yuto Katsuragawa,1,22.0,31.74,2025-07-13,Genesis Scottish Open


## Course History

In [32]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_course_history

course_hist = get_course_history(db_path, this_week_history, lookback_years=7)
course_hist_current = course_hist[this_week_key].copy()

course_hist_current.head(10)


,PLAYER,TOTAL_EVENTS_PLAYED,COURSE_HISTORY,adj_ch,ENDING_DATE,COURSE,TOURNAMENT
0,Aaron Baddeley,1,90.0,129.84,2025-07-13,The Renaissance Club,Genesis Scottish Open
1,Aaron Cockerill,1,90.0,129.84,2025-07-13,The Renaissance Club,Genesis Scottish Open
2,Aaron Rai,2,47.0,42.78,2025-07-13,The Renaissance Club,Genesis Scottish Open
3,Aaron Wise,1,5.0,7.21,2025-07-13,The Renaissance Club,Genesis Scottish Open
4,Abraham Ancer,1,14.0,20.20,2025-07-13,The Renaissance Club,Genesis Scottish Open
5,Adam Hadwin,1,34.0,49.05,2025-07-13,The Renaissance Club,Genesis Scottish Open
6,Adam Scott,3,32.3,23.30,2025-07-13,The Renaissance Club,Genesis Scottish Open
7,Adria Arnaus,2,79.0,71.91,2025-07-13,The Renaissance Club,Genesis Scottish Open
8,Adrian Meronk,1,90.0,129.84,2025-07-13,The Renaissance Club,Genesis Scottish Open
9,Adrian Otaegui,2,90.0,81.92,2025-07-13,The Renaissance Club,Genesis Scottish Open


## Merged Dataframe

In [33]:
importlib.reload(utils.db_utils)
from utils.db_utils import build_test_rows

this_week = build_test_rows(
    db_path=db_path,
    stats_df=stats_df,
    odds_df=odds_current,
    cuts_df=cuts_current,
    recent_form_df=recent_form_current,
    course_hist_df=course_hist_current,
    dk_df=dk,
    season=tournament_config["new"]["season"]
)

this_week.head()

,PLAYER,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,SEASON,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,SALARY
0,Aaron Cockerill,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,327.0,.4381,2025.0,1000.0,NaN,NaN,NaN,NaN,NaN,NaN,90.0,129.84,6100
1,Aaron Rai,27.0,0.769,22.0,0.407,46.0,0.280,64.0,0.082,117.0,-0.086,82.0,21.92%,172.0,3.19,8.0,3.97,39.0,4.56,71.0,167,166.0,287.3,1.0,74.04%,21.0,69.44%,111.0,58.77%,34.0,2.6185,2025.0,65.0,76.5,660.480,38.85,2.0,38.4,13.29,47.0,42.78,8300
2,Adam Scott,46.0,0.477,33.0,0.327,99.0,0.020,51.0,0.130,114.0,-0.075,97.0,21.53%,149.0,3.12,117.0,4.04,64.0,4.59,112.0,187,82.0,302.3,105.0,58.48%,134.0,64.35%,122.0,57.79%,42.0,2.3947,2025.0,40.0,84.6,392.203,30.17,6.0,40.2,15.23,32.3,23.30,8600
3,Adrian Otaegui,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,302.0,.4774,2025.0,1000.0,NaN,NaN,NaN,NaN,NaN,NaN,90.0,81.92,6100
4,Adrien Saddier,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,134.0,1.0914,2025.0,350.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6600


### Dataframe Normalization

Run the same normalization techniques that I ran on the historical data:
- Any NaN Odds go to 1000/1 and clip all values at 1000/1
- OWGR NaN values go to 1000
- Recent form NaN values act like a MC at 90
- Average everything else.

In [34]:
# === 1. Clean percentage stats stored as strings like '62.5%' ===
percent_stats = ["SCRAMBLING", "DRIVING_ACCURACY", "BIRDIES", "GIR"]

for col in percent_stats:
    if col in this_week.columns:
        this_week[col] = (
            this_week[col]
            .astype(str)
            .str.replace('%', '', regex=False)
            .replace(['None', 'nan', 'NaN', '--', 'DNP', ''], np.nan)
        )
        this_week[col] = pd.to_numeric(this_week[col], errors="coerce")

# === 2. Normalize & assign fallback values for key fields ===
this_week["VEGAS_ODDS"] = pd.to_numeric(this_week["VEGAS_ODDS"], errors="coerce").fillna(1000).clip(upper=1000)

if "OWGR" in this_week.columns:
    this_week["OWGR"] = pd.to_numeric(this_week["OWGR"], errors="coerce")
    max_owgr = this_week["OWGR"].dropna().max()
    this_week["OWGR"] = this_week["OWGR"].fillna(max_owgr).clip(upper=1000)

if "OWGR_RANK" in this_week.columns:
    this_week["OWGR_RANK"] = pd.to_numeric(this_week["OWGR_RANK"], errors="coerce").fillna(1000).clip(upper=1000)

this_week["RECENT_FORM"] = pd.to_numeric(this_week["RECENT_FORM"], errors="coerce").fillna(90)
this_week["FEDEX_CUP_POINTS"] = pd.to_numeric(this_week["FEDEX_CUP_POINTS"], errors="coerce").fillna(0)

if "COURSE_HISTORY" in this_week.columns:
    this_week["COURSE_HISTORY"] = pd.to_numeric(this_week["COURSE_HISTORY"], errors="coerce")
    ch_mean = this_week["COURSE_HISTORY"].mean()
    this_week["COURSE_HISTORY"] = this_week["COURSE_HISTORY"].fillna(ch_mean)

# === 3. Fill all remaining NaNs in numeric columns ===
# Recast everything that might look numeric to be sure
for col in this_week.columns:
    if col not in ["PLAYER", "SALARY", "TOURNAMENT", "SEASON"]:
        try:
            this_week[col] = pd.to_numeric(this_week[col], errors="coerce")
        except Exception:
            continue

numeric_cols = this_week.select_dtypes(include=["number"]).columns.tolist()

for col in numeric_cols:
    if this_week[col].isna().any():
        col_mean = this_week[col].mean()
        this_week[col] = this_week[col].fillna(col_mean if not np.isnan(col_mean) else 0)


In [35]:
# === Final Check: No NaNs Should Remain ===
assert this_week.isna().sum().sum() == 0, "🚨 Still missing values in prediction set!"

In [36]:
# === Check for Remaining Missing Values ===
missing_summary = this_week.isna().sum()
missing_summary = missing_summary[missing_summary > 0].sort_values(ascending=False)

print("🧹 Columns still containing NaN values:")
display(missing_summary)

🧹 Columns still containing NaN values:


Series([], dtype: int64)

In [37]:
this_week.head(10)

,PLAYER,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,SEASON,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,SALARY
0,Aaron Cockerill,67.947368,0.343711,75.25,0.1205,73.828947,0.170961,80.013158,0.052211,76.618421,0.103553,73.657895,22.309737,88.0,3.066579,64.078947,4.010132,74.105263,4.59,78.789474,167.447368,83.921053,302.281579,83.526316,60.711053,83.592105,66.382368,84.631579,59.967763,327.0,0.4381,2025.0,1000.0,62.981053,0.000,40.820737,2.494737,90.0,27.400105,90.000000,129.840000,6100
1,Aaron Rai,27.000000,0.769000,22.00,0.4070,46.000000,0.280000,64.000000,0.082000,117.000000,-0.086000,82.000000,21.920000,172.0,3.190000,8.000000,3.970000,39.000000,4.56,71.000000,167.000000,166.000000,287.300000,1.000000,74.040000,21.000000,69.440000,111.000000,58.770000,34.0,2.6185,2025.0,65.0,76.500000,660.480,38.850000,2.000000,38.4,13.290000,47.000000,42.780000,8300
2,Adam Scott,46.000000,0.477000,33.00,0.3270,99.000000,0.020000,51.000000,0.130000,114.000000,-0.075000,97.000000,21.530000,149.0,3.120000,117.000000,4.040000,64.000000,4.59,112.000000,187.000000,82.000000,302.300000,105.000000,58.480000,134.000000,64.350000,122.000000,57.790000,42.0,2.3947,2025.0,40.0,84.600000,392.203,30.170000,6.000000,40.2,15.230000,32.300000,23.300000,8600
3,Adrian Otaegui,67.947368,0.343711,75.25,0.1205,73.828947,0.170961,80.013158,0.052211,76.618421,0.103553,73.657895,22.309737,88.0,3.066579,64.078947,4.010132,74.105263,4.59,78.789474,167.447368,83.921053,302.281579,83.526316,60.711053,83.592105,66.382368,84.631579,59.967763,302.0,0.4774,2025.0,1000.0,62.981053,0.000,40.820737,2.494737,90.0,27.400105,90.000000,81.920000,6100
4,Adrien Saddier,67.947368,0.343711,75.25,0.1205,73.828947,0.170961,80.013158,0.052211,76.618421,0.103553,73.657895,22.309737,88.0,3.066579,64.078947,4.010132,74.105263,4.59,78.789474,167.447368,83.921053,302.281579,83.526316,60.711053,83.592105,66.382368,84.631579,59.967763,134.0,1.0914,2025.0,350.0,62.981053,0.000,40.820737,2.494737,90.0,27.400105,58.427451,65.226961,6600
5,Aldrich Potgieter,68.000000,0.292000,2.00,0.7300,121.000000,-0.111000,158.000000,-0.327000,55.000000,0.180000,21.000000,23.810000,97.0,3.070000,61.000000,4.010000,134.000000,4.65,30.000000,135.000000,1.000000,328.400000,134.000000,56.030000,96.000000,65.870000,172.000000,51.940000,50.0,2.1493,2025.0,90.0,38.500000,941.000,72.380000,0.000000,60.8,23.040000,58.427451,65.226961,7200
6,Alejandro Del Rey,67.947368,0.343711,75.25,0.1205,73.828947,0.170961,80.013158,0.052211,76.618421,0.103553,73.657895,22.309737,88.0,3.066579,64.078947,4.010132,74.105263,4.59,78.789474,167.447368,83.921053,302.281579,83.526316,60.711053,83.592105,66.382368,84.631579,59.967763,198.0,0.7287,2025.0,250.0,62.981053,0.000,40.820737,2.494737,90.0,27.400105,15.000000,21.640000,6400
7,Alejandro Tosti,48.000000,0.464000,27.00,0.3630,128.000000,-0.161000,24.000000,0.261000,168.000000,-0.643000,49.000000,22.560000,112.0,3.080000,61.000000,4.010000,59.000000,4.58,31.000000,136.000000,12.000000,313.600000,124.000000,56.810000,124.000000,64.890000,74.000000,60.440000,153.0,0.8950,2025.0,400.0,42.100000,316.783,16.670000,0.000000,66.4,22.160000,58.427451,65.226961,6500
8,Alex Fitzpatrick,67.947368,0.343711,75.25,0.1205,73.828947,0.170961,80.013158,0.052211,76.618421,0.103553,73.657895,22.309737,88.0,3.066579,64.078947,4.010132,74.105263,4.59,78.789474,167.447368,83.921053,302.281579,83.526316,60.711053,83.592105,66.382368,84.631579,59.967763,256.0,0.5729,2025.0,350.0,62.981053,0.000,40.820737,2.494737,90.0,27.400105,90.000000,129.840000,6500
9,Alex Noren,67.947368,0.343711,75.25,0.1205,73.828947,0.170961,80.013158,0.052211,76.618421,0.103553,73.657895,22.30973

## Correlations
**Last 4 Years**

In [38]:
import plotly.express as px

# Get the 4 most recent seasons (or fewer if not available)
available_seasons = sorted(training_df["SEASON"].unique())[-4:]

# Dictionary to store correlation results
correlation_frames = {}

# Calculate correlations for each season
for season in available_seasons:
    df = training_df[training_df["SEASON"] == season].copy()

    # Numeric features only, drop final result
    numeric = df.select_dtypes(include=["float64", "int64"]).drop(columns=["FINAL_POS", "SEASON"], errors="ignore")

    # Skip if TOP_20 is not present
    if "TOP_20" not in numeric.columns:
        continue

    # Correlation with TOP_20
    cor = numeric.corr().abs()["TOP_20"].drop("TOP_20", errors="ignore").sort_values(ascending=False)
    cor_df = cor.reset_index()
    cor_df.columns = ["Feature", "Correlation"]
    correlation_frames[season] = cor_df

# Plot one bar chart per season
for season, cor_df in correlation_frames.items():
    fig = px.bar(
        cor_df,
        x="Feature",
        y="Correlation",
        title=f"🔎 Correlation to TOP_20 — Season {int(season)}",
        height=500,
        width=1000
    )

    fig.update_layout(
        xaxis_title="Feature",
        yaxis_title="Absolute Correlation to TOP_20",
        template="plotly_dark",
        xaxis_tickangle=-45
    )

    fig.show()


**Average of Last 4 Years**

In [39]:
import pandas as pd
import numpy as np
import plotly.express as px

# === Rebuild correlation matrices for the 4 most recent seasons ===
cor = {}
recent_seasons = sorted(training_df["SEASON"].unique())[-4:]

for season in recent_seasons:
    df_season = training_df[training_df["SEASON"] == season].copy()
    numeric_cols = df_season.select_dtypes(include=["float64", "int64"])
    cor[int(season)] = numeric_cols.corr().abs()

# === Combine into one summary DataFrame ===
cor_df = pd.DataFrame()

for season in recent_seasons:
    df = cor[season].reset_index()
    if "TOP_20" not in df.columns:
        continue
    temp = df[["index", "TOP_20"]].copy()
    temp = temp.rename(columns={"TOP_20": str(season)})
    if cor_df.empty:
        cor_df = temp
    else:
        cor_df = cor_df.merge(temp, on="index", how="outer")

# Drop target and metadata rows
cor_df = cor_df[~cor_df["index"].isin(["TOP_20", "FINAL_POS", "SEASON"])]

# Compute average correlation across seasons
cor_df["AVERAGE"] = cor_df.drop(columns=["index"]).mean(axis=1)
cor_df = cor_df.sort_values(by="AVERAGE", ascending=False)

# === Plotly Bar Chart ===
fig = px.bar(
    cor_df,
    x="index",
    y="AVERAGE",
    title="Average Correlation to TOP_20 (Last 4 Seasons)",
    labels={"index": "Feature", "AVERAGE": "Avg Correlation"},
    template="plotly_dark"
)

fig.update_layout(
    xaxis_tickangle=-45,
    height=600,
    width=1000,
    showlegend=False
)

fig.show()


# ML Model

## Predictors

In [40]:
# === Define Features and Target ===
target_col = "TOP_20"

# Columns to exclude from features
exclude = [
    "PLAYER", "TOURNAMENT", "COURSE", "ENDING_DATE", "SEASON", "TOURN_ID",
    "TOP_20", "FINAL_POS"  
]

# Select numeric feature columns
feature_cols = [
    col for col in training_df.columns
    if col not in exclude and training_df[col].dtype in [np.float64, np.int64]
]

print(f"Selected {len(feature_cols)} features:\n", feature_cols)

Selected 39 features:
 ['SGTTG_RANK', 'SGTTG', 'SGOTT_RANK', 'SGOTT', 'SGAPR_RANK', 'SGAPR', 'SGATG_RANK', 'SGATG', 'SGP_RANK', 'SGP', 'BIRDIES_RANK', 'BIRDIES', 'PAR_3_RANK', 'PAR_3', 'PAR_4_RANK', 'PAR_4', 'PAR_5_RANK', 'PAR_5', 'TOTAL_DRIVING_RANK', 'TOTAL_DRIVING', 'DRIVING_DISTANCE_RANK', 'DRIVING_DISTANCE', 'DRIVING_ACCURACY_RANK', 'DRIVING_ACCURACY', 'GIR_RANK', 'GIR', 'SCRAMBLING_RANK', 'SCRAMBLING', 'OWGR_RANK', 'OWGR', 'VEGAS_ODDS', 'CUT_PERCENTAGE', 'FEDEX_CUP_POINTS', 'form_density', 'CONSECUTIVE_CUTS', 'RECENT_FORM', 'adj_form', 'COURSE_HISTORY', 'adj_ch']


### Random Forest
Initial default model before hyperparameter tuning.

This one actually performed really well (ROC AUC above 0.85).  Strangely doing hyperparameter tuning hurt the model (less than 0.7), so I removed that step and we will just use this default model.  This does cross-validation testing on 5 folds to ensure robust train/test splits.

In [41]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

# === Standardize features ===
scaler = StandardScaler()
X_rf_std = scaler.fit_transform(training_df[feature_cols])
y_rf = training_df["TOP_20"]

# === Balance the dataset with SMOTE and undersampling ===
over = SMOTE(sampling_strategy=0.5, k_neighbors=3)
under = RandomUnderSampler(sampling_strategy=0.5)
pipeline = Pipeline([("o", over), ("u", under)])
X_rf_bal, y_rf_bal = pipeline.fit_resample(X_rf_std, y_rf)

# === Define and evaluate Random Forest ===
rf = RandomForestClassifier(n_estimators=100, random_state=42)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(rf, X_rf_bal, y_rf_bal, cv=cv, scoring="roc_auc")

print("📊 Random Forest (All Features)")
print("Cross-validation scores:", scores)
print("Average ROC AUC score: {:.2f}".format(scores.mean()))


📊 Random Forest (All Features)
Cross-validation scores: [0.93496503 0.92956731 0.928125   0.85432692 0.90269886]
Average ROC AUC score: 0.91


Random Forest Training & Feature Importance

Now we train on all the data (no longer cross-validation folds) and extract the feature importance to see how it compares with linear correlations.


In [43]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.preprocessing import StandardScaler
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
import pandas as pd
import plotly.express as px

# === Resample the full dataset ===
over = SMOTE(sampling_strategy=0.5, k_neighbors=3)
under = RandomUnderSampler(sampling_strategy=0.5)
resample_pipeline = Pipeline([("o", over), ("u", under)])
X_all_resampled, y_all_resampled = resample_pipeline.fit_resample(
    training_df[feature_cols], training_df["TOP_20"]
)

# === Standardize ===
scaler = StandardScaler()
X_all_scaled = scaler.fit_transform(X_all_resampled)

# === Train Calibrated Random Forest ===
base_rf = RandomForestClassifier(random_state=42)
final_model = CalibratedClassifierCV(estimator=base_rf, method="sigmoid", cv=5)
final_model.fit(X_all_scaled, y_all_resampled)

# === Aggregate feature importances across all CV folds ===
all_importances = np.array([
    est.estimator.feature_importances_ for est in final_model.calibrated_classifiers_
])
mean_importance = all_importances.mean(axis=0)

importance_df = pd.DataFrame({
    "Feature": feature_cols,
    "Importance": mean_importance
}).sort_values(by="Importance", ascending=False)

# === Plot feature importances ===
fig = px.bar(
    importance_df,
    x="Feature",
    y="Importance",
    title="Random Forest Feature Importances",
    template="plotly_dark"
)
fig.update_layout(xaxis_tickangle=-45)
fig.show()


Use this model on this week's data to predict the best performers.

In [44]:
# === Prepare test data (this week's players) ===

# Select and standardize features using the same scaler as training
X_test = this_week[feature_cols]
X_test_scaled = scaler.transform(X_test)

# === Predict probabilities for Top 20 finish ===
# CalibratedClassifierCV returns well-calibrated probabilities
this_week["PROBABILITY"] = final_model.predict_proba(X_test_scaled)[:, 1]

# === Sort results by highest predicted probability ===
this_week_sorted = this_week.sort_values(by="PROBABILITY", ascending=False).reset_index(drop=True)

# === Define final column order explicitly ===
columns_to_show = [
    "PLAYER", "SALARY", "PROBABILITY",
    "RECENT_FORM", "COURSE_HISTORY", "VEGAS_ODDS", "CUT_PERCENTAGE", 
    "OWGR_RANK", "FEDEX_CUP_POINTS",

    # SG stats
    "SGTTG_RANK", "SGOTT_RANK", "SGAPR_RANK", "SGATG_RANK", "SGP_RANK",

    # Scoring/Performance
    "BIRDIES_RANK", "PAR_3_RANK", "PAR_4_RANK", "PAR_5_RANK",

    # Driving / Ball Striking
    "TOTAL_DRIVING_RANK", "DRIVING_DISTANCE_RANK", "DRIVING_ACCURACY_RANK",
    "GIR_RANK", "SCRAMBLING_RANK"
]

# Keep only columns that exist in the dataframe
columns_to_show = [col for col in columns_to_show if col in this_week_sorted.columns]

# Final dataframe for display/export
export_df = this_week_sorted[columns_to_show].copy()

# Preview top 20
export_df.head(20)


,PLAYER,SALARY,PROBABILITY,RECENT_FORM,COURSE_HISTORY,VEGAS_ODDS,CUT_PERCENTAGE,OWGR_RANK,FEDEX_CUP_POINTS,SGTTG_RANK,SGOTT_RANK,SGAPR_RANK,SGATG_RANK,SGP_RANK,BIRDIES_RANK,PAR_3_RANK,PAR_4_RANK,PAR_5_RANK,TOTAL_DRIVING_RANK,DRIVING_DISTANCE_RANK,DRIVING_ACCURACY_RANK,GIR_RANK,SCRAMBLING_RANK
0,J.J. Spaun,8100,0.877225,33.5,58.427451,70.0,82.600000,8.0,2254.437,13.000000,41.00,7.000000,93.000000,39.000000,59.000000,55.0,21.000000,80.000000,22.000000,61.000000,62.000000,39.000000,104.000000
1,Corey Conners,8900,0.801664,32.6,14.500000,65.0,87.500000,24.0,1465.490,28.000000,15.00,43.000000,113.000000,57.000000,78.000000,80.0,41.000000,80.000000,49.000000,141.000000,14.000000,31.000000,73.000000
2,Maverick McNealy,8000,0.750660,33.8,38.000000,50.0,82.600000,16.0,2269.672,44.000000,40.00,41.000000,125.000000,34.000000,59.000000,122.0,41.000000,64.000000,96.000000,76.000000,102.000000,48.000000,144.000000
3,Rory McIlroy,11400,0.704980,19.0,2.000000,10.0,90.900000,2.0,2944.036,11.000000,4.00,80.000000,40.000000,5.000000,13.000000,28.0,21.000000,10.000000,78.000000,2.000000,166.000000,74.000000,25.000000
4,Justin Thomas,9200,0.689789,25.1,46.700000,35.0,88.200000,4.0,2222.365,25.000000,93.00,15.000000,36.000000,12.000000,1.000000,97.0,2.000000,94.000000,137.000000,67.000000,139.000000,78.000000,109.000000
5,Collin Morikawa,9600,0.645328,23.6,3.000000,20.0,100.000000,5.0,1427.158,2.000000,10.00,5.000000,70.000000,115.000000,7.000000,6.0,21.000000,80.000000,45.000000,149.000000,3.000000,16.000000,102.000000
6,Kristoffer Reitan,6900,0.628130,90.0,58.427451,150.0,62.981053,94.0,0.000,67.947368,75.25,73.828947,80.013158,76.618421,73.657895,88.0,64.078947,74.105263,78.789474,83.921053,83.526316,83.592105,84.631579
7,Laurie Canter,6900,0.625402,81.3,90.000000,150.0,25.000000,69.0,0.000,67.947368,75.25,73.828947,80.013158,76.618421,73.657895,88.0,64.078947,74.105263,78.789474,83.921053,83.526316,83.592105,84.631579
8,Tommy Fleetwood,9900,0.619487,22.5,26.000000,20.0,92.900000,13.0,1698.648,5.000000,51.00,8.000000,17.000000,33.000000,56.000000,157.0,15.000000,59.000000,91.000000,138.000000,37.000000,128.000000,48.000000
9,Sepp Straka,8700,0.608388,34.1,90.000000,60.0,73.700000,10.0,2462.765,7.000000,37.00,3.000000,128.000000,40.000000,3.000000,1.0,21.000000,8.000000,51.000000,145.000000,11.000000,1.000000,106.000000


### Save to CSV

In [45]:
# Ensure required columns for spreadsheet export
if "COURSE_HISTORY" not in export_df.columns:
    export_df["COURSE_HISTORY"] = "-"

# Define columns to round (FedEx + stats)
columns_to_round = [
    "FEDEX_CUP_POINTS",
    "SGTTG_RANK", "SGOTT_RANK", "SGAPR_RANK", "SGATG_RANK", "SGP_RANK",
    "BIRDIES_RANK", "PAR_3_RANK", "PAR_4_RANK", "PAR_5_RANK",
    "TOTAL_DRIVING_RANK", "DRIVING_DISTANCE_RANK", "DRIVING_ACCURACY_RANK",
    "GIR_RANK", "SCRAMBLING_RANK"
]

columns_to_round_1 = [
    "COURSE_HISTORY", "CUT_PERCENTAGE"
]

# Round only the selected columns to 0 decimal places
export_df[columns_to_round] = export_df[columns_to_round].round(0).astype("Int64")
# Round only the selected columns to 1 decimal places
export_df[columns_to_round_1] = export_df[columns_to_round_1].round(1)

# Explicit column order for Excel integration
column_order = [
    "PLAYER", "SALARY", "PROBABILITY", "RECENT_FORM", "COURSE_HISTORY", "VEGAS_ODDS", 
    "CUT_PERCENTAGE", "OWGR_RANK", "FEDEX_CUP_POINTS", 
    "SGTTG_RANK", "SGOTT_RANK", "SGAPR_RANK", "SGATG_RANK", "SGP_RANK",
    "BIRDIES_RANK", "PAR_3_RANK", "PAR_4_RANK", "PAR_5_RANK",
    "TOTAL_DRIVING_RANK", "DRIVING_DISTANCE_RANK", "DRIVING_ACCURACY_RANK",
    "GIR_RANK", "SCRAMBLING_RANK"
]

# Ensure all expected columns exist
for col in column_order:
    if col not in export_df.columns:
        export_df[col] = "-" if col == "COURSE_HISTORY" else np.nan

# Reorder columns to match Excel layout
export_df = export_df[column_order]

# Export to CSV
filename = f"data/current_week_export.csv"
export_df.to_csv(filename, index=False)
print(f"✅ Exported to {filename}")

✅ Exported to data/current_week_export.csv
